<a href="https://colab.research.google.com/github/xpewa/Technopark_ML/blob/main/ML_%D0%94%D0%972.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание

В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)
 
   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

# Реализация логистической регрессии

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, 0, 0])

__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [ ]:
def probability(theta, X):
    result = 1/(1 + np.exp(-np.dot(X, theta)))
    return result
prob = probability(theta, X)

assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [ ]:
def binary_class_prediction(theta, X, threshold =.5):
    prob = probability(theta, X)
    result = (prob >= threshold).astype(int)
    return result

y_pred = binary_class_prediction(theta, X)

assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [ ]:
def logloss(theta, X, y): 
    y = np.where(y == 0 , -1, 1)
    l = X.shape[0]
    result = 1/l * np.sum(np.log(1 + np.exp(-X.dot(theta) * y)))
    y = np.where(y == -1, 0, 1)
    return result

assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

Реализуем функцию расчета градиента функционала качества в матричном виде:

In [ ]:
def gradient(theta, X, y):
    y = np.where(y == 0 , -1, 1)
    n = X.shape[1]
    result = 1/n * np.sum(1 / (1 + np.exp(-y * np.dot(X, theta))) * np.exp(-y * np.dot(X, theta)) * (-y * X.T), axis=1)
    y = np.where(y == -1, 0, 1)
    return result 

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [ ]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)
    
    errors = []
    
    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]
    
    # plt 
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
        
        
    for _ in range(iters):  
        
        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'
        
        
        # Обновляем веса
        
        theta -= grad * h
        
        theta_history = np.vstack((theta_history, theta))
        
        # error
        loss = logloss(theta, X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()   
            
    return theta

In [ ]:
X, y = make_classification(n_samples=2000)

In [ ]:
optimal_theta = fit(X, y, plot=False)

In [ ]:
y_pred = binary_class_prediction(optimal_theta, X)

Итоговый класс

In [ ]:
class MyLogisticRegression:
  def probability(self, X):
    result = 1/(1 + np.exp(-np.dot(X, self.theta)))
    return result

  def binary_class_prediction(self, X, threshold =.5):
    prob = self.probability(X)
    result = (prob >= threshold).astype(int)
    return result

  def logloss(self, X, y): 
    y = np.where(y == 0 , -1, 1)
    l = X.shape[0]
    result = 1/l * np.sum(np.log(1 + np.exp(-X.dot(theta) * y)))
    y = np.where(y == -1, 0, 1)
    return result

  def _gradient(self, X, y):
    y = np.where(y == 0 , -1, 1)
    n = X.shape[1]
    result = 1/n * np.sum(1 / (1 + np.exp(-y * np.dot(X, theta))) * np.exp(-y * np.dot(X, theta)) * (-y * X.T), axis=1)
    y = np.where(y == -1, 0, 1)
    return result 
  
  def fit(self, X, y, batch_size=10, h=0.05,  iters=100, plot=True):
    size, dim = X.shape

    self.theta = np.random.uniform(size=dim)
    errors = [] 
    theta_history = self.theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]

    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')
           
    for _ in range(iters):  
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        grad = self._gradient(X_batch, y_batch)
        
        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(self.theta), 'длина вектора должна быть равной количеству весов'
        
        self.theta -= grad * h 
        theta_history = np.vstack((theta_history, self.theta))

        loss = self.logloss(X, y)
        errors.append(loss)
        
        if plot:
            ax1.clear()            
            ax1.scatter(range(dim), self.theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')
            
            
            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')
            
            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()         

# Подготовка данных

In [ ]:
X = pd.read_csv('https://raw.githubusercontent.com/xpewa/Technopark_ML/main/weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
X.head()

,Unnamed: 0,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Unnamed: 0     142193 non-null  int64  
 1   Date           142193 non-null  object 
 2   Location       142193 non-null  object 
 3   MinTemp        141556 non-null  float64
 4   MaxTemp        141871 non-null  float64
 5   Rainfall       140787 non-null  float64
 6   Evaporation    81350 non-null   float64
 7   Sunshine       74377 non-null   float64
 8   WindGustDir    132863 non-null  object 
 9   WindGustSpeed  132923 non-null  float64
 10  WindDir9am     132180 non-null  object 
 11  WindDir3pm     138415 non-null  object 
 12  WindSpeed9am   140845 non-null  float64
 13  WindSpeed3pm   139563 non-null  float64
 14  Humidity9am    140419 non-null  float64
 15  Humidity3pm    138583 non-null  float64
 16  Pressure9am    128179 non-null  float64
 17  Pressure3pm    128212 non-nul

**Работа с признаками**

Удалим "лишние" (на мой взгляд) признаки:

* День и год в Date, т.к. на осадки влияет именно месяц
* Location, т.к. различный в тестовых и обучающих данных
* Evaporation, Sunshine - мало данных

Пустые числовые значения заполним средним. Категориальные - на NotGiven.

In [ ]:
X.RainToday = X.RainToday.replace({'No':0, 'Yes': 1})

X.Date = X.Date.apply(lambda x: x.split("-")[1])
del X['Location']
del X['Evaporation']
del X['Sunshine']
del X['Unnamed: 0']

numeric = X.select_dtypes([np.number])
X[numeric.columns] = X[numeric.columns].fillna(numeric.mean())
categorical = X.select_dtypes(exclude=[np.number])
X[categorical.columns] = X[categorical.columns].fillna("NotGiven")

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   WindGustDir    142193 non-null  object 
 5   WindGustSpeed  142193 non-null  float64
 6   WindDir9am     142193 non-null  object 
 7   WindDir3pm     142193 non-null  object 
 8   WindSpeed9am   142193 non-null  float64
 9   WindSpeed3pm   142193 non-null  float64
 10  Humidity9am    142193 non-null  float64
 11  Humidity3pm    142193 non-null  float64
 12  Pressure9am    142193 non-null  float64
 13  Pressure3pm    142193 non-null  float64
 14  Cloud9am       142193 non-null  float64
 15  Cloud3pm       142193 non-null  float64
 16  Temp9am        142193 non-null  float64
 17  Temp3pm        142193 non-nul

Разбиение на обучающую и тестовую выборку.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=1, shuffle=False)

# Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

column_transformer = ColumnTransformer([
    ('categorial', OneHotEncoder(handle_unknown="ignore"), categorical.columns),
    ('numeric', StandardScaler(), numeric.columns)
])

pipeline = Pipeline(steps=[
    ('categorial+numeric', column_transformer),
    ('classify', LogisticRegression())
])

In [ ]:
%%time
model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: user 2.2 s, sys: 20.1 ms, total: 2.22 s
Wall time: 2.23 s


In [ ]:
from sklearn.metrics import roc_auc_score, classification_report, mean_absolute_error
ROC_AUC_logreg = round(roc_auc_score(y_test, y_pred), 3)
MAE_logreg = round(mean_absolute_error(y_test, y_pred), 3)

print(classification_report(y_test, y_pred))
print("ROC_AUC: ", ROC_AUC_logreg)
print("MAE: ", MAE_logreg)

              precision    recall  f1-score   support

           0       0.87      0.96      0.91     27882
           1       0.75      0.47      0.58      7667

    accuracy                           0.85     35549
   macro avg       0.81      0.71      0.74     35549
weighted avg       0.84      0.85      0.84     35549

ROC_AUC:  0.713
MAE:  0.148


# Метод ближайших соседей

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = Pipeline(steps=[
    ('categorial+numeric', column_transformer),
    ('classify', KNeighborsClassifier())
])

In [ ]:
%%time
model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: user 4min 23s, sys: 1.99 s, total: 4min 25s
Wall time: 4min 24s


In [ ]:
ROC_AUC_kn = round(roc_auc_score(y_test, y_pred), 3)
MAE_kn = round(mean_absolute_error(y_test, y_pred), 3)

print(classification_report(y_test, y_pred))
print("ROC_AUC: ", ROC_AUC_kn)
print("MAE: ", MAE_kn)

              precision    recall  f1-score   support

           0       0.86      0.94      0.89     27882
           1       0.65      0.42      0.51      7667

    accuracy                           0.83     35549
   macro avg       0.75      0.68      0.70     35549
weighted avg       0.81      0.83      0.81     35549

ROC_AUC:  0.681
MAE:  0.173


# Байесовский классификатор

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.base import TransformerMixin

class DenseTransformer(TransformerMixin):
  
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

pipeline = Pipeline(steps=[
    ('categorial+numeric', column_transformer),
    ('to_dense', DenseTransformer()), 
    ('classify', GaussianNB())
])

In [ ]:
%%time
model = pipeline.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: user 576 ms, sys: 77.2 ms, total: 653 ms
Wall time: 663 ms


In [ ]:
ROC_AUC_nb = round(roc_auc_score(y_test, y_pred), 3)
MAE_nb = round(mean_absolute_error(y_test, y_pred), 3)

print(classification_report(y_test, y_pred))
print("ROC_AUC: ", ROC_AUC_nb)
print("MAE: ", MAE_nb)

              precision    recall  f1-score   support

           0       0.90      0.82      0.86     27882
           1       0.51      0.67      0.58      7667

    accuracy                           0.79     35549
   macro avg       0.70      0.74      0.72     35549
weighted avg       0.82      0.79      0.80     35549

ROC_AUC:  0.745
MAE:  0.212


# Моя логистическая регрессия

In [ ]:
X_train_array = column_transformer.fit_transform(X_train).toarray()
X_test_array = column_transformer.fit_transform(X_test).toarray()

In [ ]:
%%time
optimal_theta = fit(X_train_array, y_train, plot=False)
y_pred = binary_class_prediction(optimal_theta, X_test_array)

CPU times: user 3.66 s, sys: 1.41 s, total: 5.07 s
Wall time: 2.66 s


In [ ]:
ROC_AUC_mylogreg = round(roc_auc_score(y_test, y_pred), 3)
MAE_mylogreg = round(mean_absolute_error(y_test, y_pred), 3)

print(classification_report(y_test, y_pred))
print("ROC_AUC: ", ROC_AUC_mylogreg)
print("MAE: ", MAE_mylogreg)

              precision    recall  f1-score   support

           0       0.96      0.24      0.38     27882
           1       0.26      0.96      0.41      7667

    accuracy                           0.40     35549
   macro avg       0.61      0.60      0.40     35549
weighted avg       0.81      0.40      0.39     35549

ROC_AUC:  0.602
MAE:  0.604


# Выводы

Сравним результаты работы моделей по качеству и времени выполнения.


In [ ]:
results = [['LogisticRegression', 2.23, 0.85, ROC_AUC_logreg, MAE_logreg],
           ['KNeighborsClassifier', 264, 0.83, ROC_AUC_kn, MAE_kn],
           ['Naive Bayes', 0.663, 0.79, ROC_AUC_nb, MAE_nb],
           ['MyLogisticRegression', 2.66, 0.40, ROC_AUC_mylogreg, MAE_mylogreg]]

results_df = pd.DataFrame(results, columns=['Model', 'Time (s)', 'Accuracy', 'ROC_AUC', 'MAE'])
results_df

,Model,Time (s),Accuracy,ROC_AUC,MAE
0,LogisticRegression,2.230,0.85,0.713,0.148
1,KNeighborsClassifier,264.000,0.83,0.681,0.173
2,Naive Bayes,0.663,0.79,0.745,0.212
3,MyLogisticRegression,2.660,0.40,0.602,0.604


Наиболее быстрый - Байесовский классификатор. 
Время выполнения обоих логистических регрессий соизмеримо, но самостоятельно реализованная медленнее. 
Худший результат с большим отрывом показал метод ближайших соседей.

По доле правильных ответов (accuracy) лидирует логистическая регрессия.
Площадь под кривой ошибок (ROC_AUC) большая у Байесовского классификатора.
И средняя абсолютная ошибка (MAE) наименьшая в случае логистической регрессии.

Худшей по всем параметрам оказалась логистическая регрессия, реализованная самостоятельно.

Считаю, что победитель - логистическая регрессия.